1. conda install all strox dependencies
2. use jupyter not colab 
3. add conda env package list to commit
4. ok imports

In [67]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.etree.ElementTree as ET
from lxml import etree
import warnings

In [68]:
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from scipy.sparse import hstack

In [69]:
home = '/opt/anaconda3/stanford-postagger-full-2020-08-06'
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
from nltk import word_tokenize
_path_to_model = home + '/models/english-bidirectional-distsim.tagger' 
_path_to_jar = home + '/stanford-postagger.jar'
stanford_tag = POS_Tag(model_filename=_path_to_model, path_to_jar=_path_to_jar)

In [70]:
path_train = 'ABSA16_Laptops_Train_English_SB2.xml'
path_test = 'EN_LAPT_SB2_TEST.xml.gold'

so [joblib](https://joblib.readthedocs.io/en/latest/index.html) is python pipeline processing lib

sklearn is used here. this is a maintained and reliable library according to [this guy](https://www.youtube.com/watch?v=rvVkVsG49uU)

- get 
    - ABSA16_Laptops_Train_English_SB2.xml 
    - EN_LAPT_SB2_TEST.xml
    - stanford pos tagger stuff
    - tagged_text_list_train.pkl files
- add function definitions

In [71]:
def get_list(path):
    tree=ET.parse(path)
    root = tree.getroot()
    text_list = []
    opinion_list = []
    for review in root.findall('Review'):
        text_string=""
        opinion_inner_list=[]
        for sent in review.findall('./sentences/sentence'):
            text_string= text_string+ " "+ sent.find('text').text
        text_list.append(text_string)
        for opinion in review.findall('./Opinions/Opinion'):
            opinion_dict = {
                opinion.get('category').replace('#','_'): opinion.get('polarity')
            }
            opinion_inner_list.append(opinion_dict)
        opinion_list.append(opinion_inner_list)
    return text_list,opinion_list

In [72]:
def get_most_common_aspect(opinion_list):
    import nltk
    opinion= []
    for inner_list in opinion_list:
        for _dict in inner_list:
            for key in _dict:
                opinion.append(key)
    most_common_aspect = [k for k,v in nltk.FreqDist(opinion).most_common(20)]
    return most_common_aspect

In [73]:
def get_data_frame(text_list,opinion_list,most_common_aspect):
    data={'Review':text_list}
    df = pd.DataFrame(data)
    if opinion_list:
        for inner_list in opinion_list:
            for _dict in inner_list:
                for key in _dict:
                    if key in most_common_aspect:
                        df.loc[opinion_list.index(inner_list),key]=_dict[key]
    return df

In [74]:
def get_aspect_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive','negative','neutral','conflict'],[1,1,1,1])
    df = df.fillna(0)
    return df

In [75]:
def get_aspect_data_frame_test(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        if df.get(common_aspect,0):
            df[common_aspect]=df[common_aspect].replace(['positive','negative','neutral','conflict'],[1,1,1,1])
    df = df.fillna(0)
    return df

In [76]:
def get_positive_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive'],[1])
        df[common_aspect]=df[common_aspect].replace(['negative','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df

In [77]:
def get_negative_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['negative'],[1])
        df[common_aspect]=df[common_aspect].replace(['positive','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df

In [78]:
def get_neutral_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['neutral','conflict'],[1,1])
        df[common_aspect]=df[common_aspect].replace(['negative','positive'],[0,0])
    df = df.fillna(0)
    return df

In [79]:
def posTag(review):
    tagged_text_list=[]
    for text in review:
        tagged_text_list.append(stanford_tag.tag(word_tokenize(text)))
    return tagged_text_list

In [80]:
def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list

In [81]:
def get_dict_aspect(y,most_common_aspect):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(most_common_aspect)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect

In [82]:
def get_sample(l1,l2=None,n=1):
    for i in range(0,n):
        rand_int = int(np.random.rand() * len(l1))
        if l2:
            print(l1[rand_int]), '\n\n', print(l2[rand_int])
        else:
            print(l1[rand_int])
            return l1[rand_int]

In [83]:
def get_meta(l):
    print('type: ',type(l))
    print('len: ',len(l))

**[thestrox Github code](https://github.com/thestrox/Aspect-Based-Sentiment-Analysis/blob/master/Aspect%20Based%20Sentiment%20Analysis.ipynb)**

In [84]:
train_text_list, train_opinion_list = get_list(path_train)
# get_sample(train_text_list,train_opinion_list,n=2)

In [85]:
most_common_aspect = get_most_common_aspect(train_opinion_list)
# most_common_aspect

In [86]:
tagged_text_list_train=joblib.load('tagged_text_list_train.pkl')

In [87]:
final_train_text_list = filterTag(tagged_text_list_train)
# get_sample(tagged_text_list_train,final_train_text_list)

Each train review and its SB2 (aspect,sentiment) pair is POS-tagged, and only nouns, adjectives, verbs and adverbs are retained according to [Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) POS table.

In [88]:
df_train = get_data_frame(train_text_list,train_opinion_list,most_common_aspect)
# df_train.sample(3)
# df_train.describe()
# df_train.columns

In [89]:
df_train_aspect = get_aspect_data_frame(df_train,most_common_aspect)
# df_train_aspect

Repeat for test data.

In [90]:
test_text_list,test_opinion_list = get_list(path_test)

In [91]:
tagged_text_list_test=joblib.load('tagged_text_list_test.pkl')

In [92]:
final_test_text_list=filterTag(tagged_text_list_test)

In [93]:
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)
df_test_aspect = get_aspect_data_frame(df_test,most_common_aspect)
df_test_aspect = df_test_aspect.reindex(sorted(df_test_aspect.columns), axis=1)

In [94]:
X_train= df_train_aspect.Review
y_train = df_train_aspect.drop('Review',1)

In [95]:
X_test = df_test_aspect.Review
y_test = df_test_aspect.drop('Review',1)

In [96]:
import numpy as np
y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
vect = CountVectorizer(max_df=1.0,stop_words='english')  
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [98]:
nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
C = 1.0 #SVregularization parameter
svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)

In [99]:
y_pred_class = nb_classif.predict(X_test_dtm)
y_pred_class_svc = svc.predict(X_test_dtm)
y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
y_pred_class_sgd = sgd.predict(X_test_dtm)

In [100]:
from sklearn import metrics

In [101]:
print(metrics.accuracy_score(y_test,y_pred_class))
print(metrics.accuracy_score(y_test,y_pred_class_svc))
print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
print(metrics.accuracy_score(y_test,y_pred_class_sgd))

0.0
0.0
0.0
0.0


In [102]:
print(metrics.precision_score(y_test,y_pred_class,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))

0.22672064777327935
0.24598930481283424
0.2349570200573066
0.22968197879858657


In [103]:
print(metrics.recall_score(y_test,y_pred_class,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))

0.13559322033898305
0.22276029055690072
0.19854721549636803
0.15738498789346247


In [104]:
print(metrics.f1_score(y_test,y_pred_class,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))

0.1696969696969697
0.23379923761118174
0.2152230971128609
0.18678160919540232


In [105]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

              precision    recall  f1-score   support

           0       0.17      1.00      0.30        14
           1       0.22      0.46      0.30        24
           2       0.11      0.17      0.13        12
           3       0.00      0.00      0.00         4
           4       0.45      0.71      0.56        21
           5       0.06      0.12      0.08         8
           6       0.20      0.43      0.27         7
           7       0.14      0.05      0.08        39
           8       1.00      0.06      0.12        80
           9       1.00      0.04      0.08        24
          10       1.00      0.02      0.04        46
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00        27
          13       0.00      0.00      0.00        29
          14       0.00      0.00      0.00        30
          15       0.12      0.25      0.17         4
          16       0.00      0.00      0.00         9
          17       0.00    

In [106]:
train_dict_aspect=get_dict_aspect(y_train, most_common_aspect)
d_train=DictVectorizer() 
X_train_aspect_dtm = d_train.fit_transform(train_dict_aspect)

In [107]:
test_dict_aspect=get_dict_aspect(y_test,most_common_aspect)
d_test=DictVectorizer() 
X_test_aspect_dtm = d_test.fit_transform(test_dict_aspect)

In [108]:
def classify_sentiment(df_train,df_test,X_train_aspect_dtm,X_test_aspect_dtm):
    
    df_train = df_train.reindex(sorted(df_train_positive.columns), axis=1)
    df_test = df_test.reindex(sorted(df_test_positive.columns), axis=1)

    import numpy as np
    X_train = df_train.Review
    y_train = df_train.drop('Review',1)
    y_train = np.asarray(y_train, dtype=np.int64)

    X_test = df_test.Review
    y_test = df_test.drop('Review',1)
    y_test = np.asarray(y_test, dtype=np.int64)

    vect_sen = CountVectorizer(stop_words='english',ngram_range=(1,2))  
    X_train_dtm = vect_sen.fit_transform(X_train)
    X_test_dtm = vect_sen.transform(X_test)

    #ombining word vector with extra feature.
    from scipy.sparse import hstack
    X_train_dtm=hstack((X_train_dtm, X_train_aspect_dtm))
    X_test_dtm=hstack((X_test_dtm, X_test_aspect_dtm))

    C = 1.0 #SVregularization parameter
    nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
    svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
    lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
    sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)

    y_pred_class= nb_classif.predict(X_test_dtm)
    y_pred_class_svc = svc.predict(X_test_dtm)
    y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
    y_pred_class_sgd = sgd.predict(X_test_dtm)
    return (y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd)

In [109]:
def print_metrices(y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd):
    print("Accuracy:")
    print(metrics.accuracy_score(y_test,y_pred_class))
    print(metrics.accuracy_score(y_test,y_pred_class_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_sgd))

    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred_class,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred_class,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))
    
    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred_class,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

In [110]:
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
df_test_positive = get_positive_data_frame(df_test,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test_positive,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos)

Accuracy:
0.1
0.0
0.0
0.0

Average precision:
0.821917808219178
0.23737373737373738
0.21764705882352942
0.24394463667820068

Average recall:
0.21739130434782608
0.17028985507246377
0.26811594202898553
0.5108695652173914

Average f1:
0.3438395415472779
0.19831223628691982
0.24025974025974028
0.33021077283372363

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.00      0.00      0.00        14
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        15
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00        26
           8       0.82      0.95      0.88        61
           9       0.00      0.00      0.00        11
          10       1.00      0.06      0.11        36
          11   

In [111]:
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_neg = get_negative_data_frame(df_train,most_common_aspect)
df_test_neg = get_negative_data_frame(df_test,most_common_aspect)

y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_neg,df_test_neg,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg)

Accuracy:
0.5
0.25
0.0625
0.0375

Average precision:
0.8
0.09090909090909091
0.10227272727272728
0.0764872521246459

Average recall:
0.03669724770642202
0.07339449541284404
0.24770642201834864
0.24770642201834864

Average f1:
0.07017543859649122
0.08121827411167513
0.14477211796246647
0.11688311688311688

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         3
           8       0.80      0.22      0.35        18
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        10
          11       0.

In [112]:
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_neu = get_neutral_data_frame(df_train,most_common_aspect)
df_test_neu = get_neutral_data_frame(df_test,most_common_aspect)

y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu=classify_sentiment(df_train_neu,df_test_neu,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu)

Accuracy:
0.7375
0.6875
0.275
0.125

Average precision:
0.0
0.1
0.028985507246376812
0.007575757575757576

Average recall:
0.0
0.03571428571428571
0.07142857142857142
0.03571428571428571

Average f1:
0.0
0.05263157894736841
0.04123711340206185
0.0125

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00        10
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.0

In [113]:
user_input="This is my first asus laptop. So far i am really enjoying this laptop. 512GB SSD is super fast. Battery life is also good and can last very long. I have no complain on screen quality too as display supports 4k videos. Maybe that is why it costs a lot. This is an expensive laptop and it's price is very high compared to other laptops of similar specs. So, if you have no trouble paying for this laptop, it is pretty good."
tagged_user_input = posTag([user_input])
filter_tagged_user_input = filterTag(tagged_user_input)

user_input_series=pd.Series(filter_tagged_user_input)
user_input_series_dtm=vect.transform(user_input_series)

predict_aspect= svc.predict(user_input_series_dtm)
extra_feature=get_dict_aspect(predict_aspect, most_common_aspect)
extra_feature_dtm=DictVectorizer().fit_transform(extra_feature)
predict_aspect

array([[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]])

In [114]:
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_pos

array([[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [115]:
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_negative = get_negative_data_frame(df_train,most_common_aspect)
y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_negative,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_neg

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [116]:
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_neutral = get_neutral_data_frame(df_train,most_common_aspect)
y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu=classify_sentiment(df_train_neutral,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_neu

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [117]:
index_positive=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_pos.tolist()[0])):
    if a ==1 and b==1:
        index_positive.append(i)
index_positive

[0]

In [118]:
index_negative=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_neg.tolist()[0])):
    if a ==1 and b==1:
        index_negative.append(i)
index_negative

[]

In [119]:
index_neutral=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_neu.tolist()[0])):
    if a ==1 and b==1:
        index_neutral.append(i)
index_neutral

[]

In [120]:
output=[]

In [121]:
if index_positive:
    for index in index_positive:
        output.append(sorted(most_common_aspect)[index]+": positive")

In [122]:
if index_negative:
    for index in index_negative:
        output.append(sorted(most_common_aspect)[index]+": negative")

In [123]:
if index_neutral:
    for index in index_neutral:
        output.append(sorted(most_common_aspect)[index]+": neutral or conflict")

In [124]:
output

['BATTERY_OPERATION_PERFORMANCE: positive']

# Rule-based aspects output